In [30]:
import pandas as pd
import numpy as np
import pandasql as psql
from stockstats import StockDataFrame as Sdf

# Load the dataset
file_path = '../data/raw/indexProcessed.csv' 
file_path2 = '../data/raw/indexInfo.csv'

df = pd.read_csv(file_path)
df2 = pd.read_csv(file_path2)
# Drop the 'currency' column from df2
df2 = df2.drop(columns=['Currency'])
df = pd.merge(df, df2, on='Index', how='left')
print("merge")
print (df.head())
query = """
SELECT * 
FROM df
WHERE Open IS NOT NULL
AND Close IS NOT NULL
AND Volume IS NOT NULL
"""

# Execute the query
df_cleaned = psql.sqldf(query, locals())


# 1. Drop rows with any missing (NaN) values
#df_cleaned = df.dropna()

# Define thresholds for outliers based on the 1st and 99th percentiles for each feature

volume_threshold = (df_cleaned['Volume'].quantile(0.01), df_cleaned['Volume'].quantile(0.99))
open_threshold = (df_cleaned['Open'].quantile(0.01), df_cleaned['Open'].quantile(0.99))
high_threshold = (df_cleaned['High'].quantile(0.01), df_cleaned['High'].quantile(0.99))
low_threshold = (df_cleaned['Low'].quantile(0.01), df_cleaned['Low'].quantile(0.99))
close_threshold = (df_cleaned['Close'].quantile(0.01), df_cleaned['Close'].quantile(0.99))
adj_close_threshold = (df_cleaned['Adj Close'].quantile(0.01), df_cleaned['Adj Close'].quantile(0.99))

# Remove rows where 'Volume', 'Open', 'High', 'Low', 'Close', or 'Adj Close' fall outside of these thresholds
df_cleaned = df_cleaned[
    (df_cleaned['Volume'] >= volume_threshold[0]) & (df_cleaned['Volume'] <= volume_threshold[1]) &
    (df_cleaned['Open'] >= open_threshold[0]) & (df_cleaned['Open'] <= open_threshold[1]) &
    (df_cleaned['High'] >= high_threshold[0]) & (df_cleaned['High'] <= high_threshold[1]) &
    (df_cleaned['Low'] >= low_threshold[0]) & (df_cleaned['Low'] <= low_threshold[1]) &
    (df_cleaned['Close'] >= close_threshold[0]) & (df_cleaned['Close'] <= close_threshold[1]) &
    (df_cleaned['Adj Close'] >= adj_close_threshold[0]) & (df_cleaned['Adj Close'] <= adj_close_threshold[1])
]
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

# 3. Remove Duplicates (if any)
df_cleaned = df_cleaned.drop_duplicates()

# 4. (Optional) Filter rows based on a specific date range (assuming there's a 'Date' column)
# Convert 'Date' column to datetime if needed

# 5. Example of creating a new feature (similar to `CloseUSD`)
# Let's assume we are converting 'Close' price to USD using a fixed exchange rate
#usd_conversion_rate = 0.85  # Example conversion rate
#df_cleaned['CloseUSD'] = df_cleaned['Close'] * usd_conversion_rate

# Create daily price difference
df_cleaned['price_diff'] = df_cleaned['Close'] - df_cleaned['Open']

# Calculate moving averages for closing prices (5-day, 10-day, 30-day)
# df_cleaned['ma_5'] = df_cleaned['Close'].rolling(window=5).mean()
# df_cleaned['ma_10'] = df_cleaned['Close'].rolling(window=10).mean()
# df_cleaned['ma_30'] = df_cleaned['Close'].rolling(window=30).mean()

df_cleaned['ma_5'] = df_cleaned.groupby('Index')['Close'].rolling(window=5).mean().reset_index(level=0, drop=True)
df_cleaned['ma_10'] = df_cleaned.groupby('Index')['Close'].rolling(window=10).mean().reset_index(level=0, drop=True)
df_cleaned['ma_30'] = df_cleaned.groupby('Index')['Close'].rolling(window=30).mean().reset_index(level=0, drop=True)



# Calculate daily volatility using high and low prices
df_cleaned['volatility'] = df_cleaned['High'] - df_cleaned['Low']

# Calculate percentage returns
df_cleaned['pct_return'] = df_cleaned['Close'].pct_change()

# Drop the first row due to NaN values created by pct_change
df_cleaned = df_cleaned.dropna()




# Create a target variable: 1 if next day's closing price increases, 0 otherwise
df_cleaned['price_direction'] = (df_cleaned['Close'].shift(-1) > df_cleaned['Close']).astype(int)


#df_cleaned['Up_Or_Down'] = np.where (df_cleaned['Close'].shift(-1) > df_cleaned['Close'],1,0)


# Drop the last row since it won't have a target value
#data = data[:-1]
print(df_cleaned.columns)

# Display the new features




df_cleaned = df_cleaned.rename(columns={'Index': 'index_stock'})
print(df_cleaned.columns)
date_col = df_cleaned['Date'].copy()
num_rows = len(df_cleaned)
print(f"Number of rows: {num_rows}")


df_cleaned.to_csv('../data/processed/indexData_processed_before_calculation.csv', index=False)


stock = Sdf.retype(df_cleaned)


#Moving Average Convergence Divergence
df_cleaned['macd']=stock['macd']
df_cleaned['rsi_12']=stock['rsi_12']
#df_cleaned['cci_10']=stock['cci_10']
#df_cleaned['wr_6'] = stock['wr_6']
df_cleaned['close_5_sma']=stock['close_5_sma']
df_cleaned['close_10_ema'] = stock['close_10_ema']


# Remove rows with zeros in specified columns
df_cleaned = df_cleaned[(df_cleaned[['macd', 'rsi_12', 'close_5_sma', 'close_10_ema', 'volume']] != 0).all(axis=1)]


# Show the result
print(df_cleaned.columns)
num_rows = len(df_cleaned)
print(f"Number of rows: {num_rows}")
# Save the cleaned dataset if needed
df_cleaned.to_csv('../data/processed/indexData_processed.csv', index=False)


merge
  Index        Date         Open         High          Low        Close  \
0   HSI  1986-12-31  2568.300049  2568.300049  2568.300049  2568.300049   
1   HSI  1987-01-02  2540.100098  2540.100098  2540.100098  2540.100098   
2   HSI  1987-01-05  2552.399902  2552.399902  2552.399902  2552.399902   
3   HSI  1987-01-06  2583.899902  2583.899902  2583.899902  2583.899902   
4   HSI  1987-01-07  2607.100098  2607.100098  2607.100098  2607.100098   

     Adj Close  Volume    CloseUSD     Region                  Exchange  
0  2568.300049     0.0  333.879006  Hong Kong  Hong Kong Stock Exchange  
1  2540.100098     0.0  330.213013  Hong Kong  Hong Kong Stock Exchange  
2  2552.399902     0.0  331.811987  Hong Kong  Hong Kong Stock Exchange  
3  2583.899902     0.0  335.906987  Hong Kong  Hong Kong Stock Exchange  
4  2607.100098     0.0  338.923013  Hong Kong  Hong Kong Stock Exchange  
Index(['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'CloseUSD', '